In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import kagglehub

/home/stoffl/Documents/WineRecommender/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
path = kagglehub.dataset_download("zynicide/wine-reviews")
print(path)
df = pd.read_csv(os.path.join(path, "winemag-data-130k-v2.csv"))

df

/home/stoffl/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4


IsADirectoryError: [Errno 21] Is a directory: '/home/stoffl/.cache/kagglehub/datasets/zynicide/wine-reviews/versions/4'

In [ ]:
print(df.isnull().sum())

Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64


## Data Cleaning

In [ ]:
def clean_wine_data(df):
    """
    Complete wine data cleaning pipeline
    """
    # Create a copy to avoid modifying original data
    df_clean = df.copy()
    
    print("Initial data count:", len(df_clean))
    
    # 1. Handle minor missing values (direct removal)
    df_clean = df_clean.dropna(subset=['variety'])  # Only 1 record
    
    # 2. Handle price missing values (hierarchical imputation)
    print("Handling missing prices...")
    df_clean['price'] = df_clean.groupby(['variety', 'country'])['price'].transform(
        lambda x: x.fillna(x.median()) if not x.isnull().all() else x
    )
    global_price_median = df_clean['price'].median()
    df_clean['price'] = df_clean['price'].fillna(global_price_median)
    
    # 3. Handle geographic information missing values
    print("Handling missing geographic data...")
    df_clean['country'] = df_clean['country'].fillna('Unknown_Country')
    df_clean['province'] = df_clean['province'].fillna('Unknown_Province')
    df_clean['region_1'] = df_clean['region_1'].fillna(df_clean['province'] + '_Region')
    
    # 4. Handle taster information
    df_clean['taster_name'] = df_clean['taster_name'].fillna('Unknown_Taster')
    
    # 5. Create core features
    print("Creating features...")
    df_clean['structured_features'] = (
        df_clean['variety'] + ' ' + 
        df_clean['country'] + ' ' + 
        df_clean['region_1']
    )
    
    # Price categorization
    def price_category(price):
        if price < 25: return 'budget'
        elif price < 50: return 'mid_range' 
        else: return 'premium'
    
    df_clean['price_category'] = df_clean['price'].apply(price_category)
    
    print("Cleaning completed! Final data count:", len(df_clean))
    return df_clean

# Execute cleaning
df_cleaned = clean_wine_data(df)

# Verify cleaning results
print("\n=== Missing values after cleaning ===")
print(df_cleaned.isnull().sum())

Initial data count: 129971
Handling missing prices...
Handling missing geographic data...
Creating features...
Cleaning completed! Final data count: 129970

=== Missing values after cleaning ===
Unnamed: 0                   0
country                      0
description                  0
designation              37464
points                       0
price                        0
province                     0
region_1                     0
region_2                 79459
taster_name                  0
taster_twitter_handle    31212
title                        0
variety                      0
winery                       0
structured_features          0
price_category               0
dtype: int64


## Feature Engineering

In [ ]:
def create_enhanced_features(df):
    """
    Create enhanced features with better weighting strategy
    """
    print("Creating enhanced features...")
    
    # extract flavor features
    def extract_flavor_features(description):
        if pd.isna(description):
            return ""
        
        description_lower = description.lower()
        flavor_terms = []
        
        # define flavor keywords
        flavor_keywords = {
            'fruit': ['berry', 'cherry', 'apple', 'citrus', 'tropical', 'fruit', 'blackberry', 'raspberry'],
            'dry': ['dry', 'crisp', 'tannic'],
            'sweet': ['sweet', 'honey', 'ripe', 'jam'],
            'oak': ['oak', 'vanilla', 'toast', 'cedar'],
            'spice': ['spice', 'pepper', 'cinnamon', 'clove'],
            'herbal': ['herbal', 'grass', 'mineral', 'earth']
        }
        
        # check flavor type in description
        for flavor_type, keywords in flavor_keywords.items():
            if any(keyword in description_lower for keyword in keywords):
                flavor_terms.append(flavor_type)
        
        return ' '.join(flavor_terms)
    
    # apply
    df['flavor_profile'] = df['description'].apply(extract_flavor_features)
    
    # feature Engineering
    df['enhanced_features'] = (
        "variety_" + df['variety'] + ' ' +
        "country_" + df['country'] + ' ' +
        "region_" + df['region_1'] + ' ' +
        "pricecat_" + df['price_category'] + ' ' +
        "winery_" + df['winery'] + ' ' +
        "flavor_" + df['flavor_profile']
    )
    
    # Create TF-IDF features with optimized parameters
    vectorizer = TfidfVectorizer(
        max_features=1500,
        stop_words='english',
        lowercase=True,
        min_df=8,
        max_df=0.6,
        ngram_range=(1, 2),
        analyzer='word'
    )
    
    feature_vectors = vectorizer.fit_transform(df['enhanced_features'])
    
    # Analyze feature learning
    feature_names = vectorizer.get_feature_names_out()
    variety_features = [f for f in feature_names if f.startswith('variety_')]
    country_features = [f for f in feature_names if f.startswith('country_')]
    region_features = [f for f in feature_names if f.startswith('region_')]
    flavor_features = [f for f in feature_names if f.startswith('flavor_')]
    
    print(f"Enhanced feature matrix: {feature_vectors.shape}")
    print(f"Learned {len(variety_features)} variety features")
    print(f"Learned {len(country_features)} country features") 
    print(f"Learned {len(region_features)} region features")
    print(f"Learned {len(flavor_features)} flavor features")
    
    # Show sample features to verify quality
    print(f"Sample variety features: {variety_features[:6]}")
    print(f"Sample country features: {country_features[:4]}")
    print(f"Sample flavor features: {flavor_features}")
    
    
    print(f"\nFlavor Profile Distribution:")
    for flavor_type in ['fruit', 'dry', 'sweet', 'oak', 'spice', 'herbal']:
        count = df[df['flavor_profile'].str.contains(flavor_type, na=False)].shape[0]
        percentage = (count / len(df)) * 100
        print(f"   {flavor_type}: {count} wines ({percentage:.1f}%)")
    
    return feature_vectors, vectorizer


feature_vectors, vectorizer = create_enhanced_features(df_cleaned)


🚀 Creating enhanced features...
✅ Enhanced feature matrix: (129970, 1500)
📊 Learned 175 variety features
📊 Learned 207 country features
📊 Learned 423 region features
🍷 Learned 21 flavor features
🔍 Sample variety features: ['variety_aglianico', 'variety_aglianico country_italy', 'variety_albariño', 'variety_albariño country_spain', 'variety_albariño country_us', 'variety_alvarinho']
🔍 Sample country features: ['country_argentina', 'country_argentina region_luján', 'country_argentina region_mendoza', 'country_argentina region_tupungato']
🔍 Sample flavor features: ['flavor_', 'flavor_dry', 'flavor_dry herbal', 'flavor_dry oak', 'flavor_dry spice', 'flavor_dry sweet', 'flavor_fruit dry', 'flavor_fruit herbal', 'flavor_fruit oak', 'flavor_fruit spice', 'flavor_fruit sweet', 'flavor_herbal', 'flavor_oak', 'flavor_oak herbal', 'flavor_oak spice', 'flavor_spice', 'flavor_spice herbal', 'flavor_sweet', 'flavor_sweet herbal', 'flavor_sweet oak', 'flavor_sweet spice']

📈 Flavor Profile Distributi

In [ ]:
import os
import pickle

# Create output directory if it doesn't exist
output_dir = '../models/data'
os.makedirs(output_dir, exist_ok=True)

# Save cleaned dataframe
df_cleaned.to_csv(os.path.join(output_dir, 'cleaned_wine_data.csv'), index=False)
print(f"Saved cleaned data to {output_dir}/cleaned_wine_data.csv")

# Save feature vectors (sparse matrix)
with open(os.path.join(output_dir, 'feature_vectors.pkl'), 'wb') as f:
    pickle.dump(feature_vectors, f)
print(f"Saved feature vectors to {output_dir}/feature_vectors.pkl")

# Save vectorizer
with open(os.path.join(output_dir, 'vectorizer.pkl'), 'wb') as f:
    pickle.dump(vectorizer, f)
print(f"Saved vectorizer to {output_dir}/vectorizer.pkl")

print("\nAll data saved successfully!")
